In [4]:
import pandas as pd
import numpy as np
import requests
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [6]:
response = requests.get(url)
soup = BeautifulSoup(response.text,'lxml')
table = soup.find_all('table')[0]
column_header = [th.getText() for th in soup.findAll('tr',limit=1)[0].findAll('th')]
column_header[-1] = column_header[-1].strip()
rows = table.findAll('tr')[1:]
row_data = [[td.getText() for td in rows[i].findAll('td')]for i in range (len(rows))]
df = pd.DataFrame(row_data,columns = column_header)
df.Neighbourhood = df.Neighbourhood.map(lambda x : str(x)[:-1])

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
import Folium

ImportError: No module named 'Folium'

In [8]:
dataframe = df[df.Borough != 'Not assigned']
dataframe["Neighbourhood"] = np.where(dataframe.Neighbourhood == 'Not assigned',dataframe.Borough,dataframe.Neighbourhood)
dataframe.reset_index(inplace = True)
dataframe = dataframe.drop(['index'],axis = 1)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [9]:
df2 = dataframe.groupby(['Postcode','Borough'],sort = False)['Neighbourhood'].apply(lambda x : ', '.join(x))
df3 = pd.DataFrame(df2).reset_index(inplace =False)
df3[0:5]

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [10]:
df3.shape

(103, 3)